<a href="https://colab.research.google.com/github/sharanya012/AI-powered-Multimodal-Knowledge-System-for-Skin-Diseases/blob/main/gen_ai_working_code_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RUN THIS: Second Agentic Workflow with Qlora


## Dependencies


In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
kmader_skin_cancer_mnist_ham10000_path = kagglehub.dataset_download('kmader/skin-cancer-mnist-ham10000')

print('Data source import complete.')
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
      x = os.path.join(dirname, filename)
      if filename.endswith('.csv'):
        print(x)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Using Colab cache for faster access to the 'skin-cancer-mnist-ham10000' dataset.
Data source import complete.
/kaggle/input/skin-cancer-mnist-ham10000/hmnist_8_8_RGB.csv
/kaggle/input/skin-cancer-mnist-ham10000/hmnist_28_28_RGB.csv
/kaggle/input/skin-cancer-mnist-ham10000/hmnist_8_8_L.csv
/kaggle/input/skin-cancer-mnist-ham10000/hmnist_28_28_L.csv
/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv


In [ ]:
!pip install torch transformers peft datasets faiss-cpu sentence-transformers langchain langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install faiss-cpu --quiet

In [ ]:
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==1

In [ ]:
!pip install -U langchain --quiet
!pip install -U langchain-community --quiet
!pip install -qU langgraph==0.2.45 langchain-google-genai==2.0.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.3 MB/s eta 0:00:00


In [ ]:
!pip install efficientnet-pytorch Pillow --quiet

  Preparing metadata (setup.py) ... done


## Agentic workflow


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import faiss
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AIMessage, HumanMessage
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel,
    TaskType
)
from datasets import Dataset

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cpu


In [ ]:
# Load TinyLlama model
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Use TinyLlama chat model

print(f"Loading model: {model_name}")

Loading model: TinyLlama/TinyLlama-1.1B-Chat-v1.0


In [ ]:
# Define image preprocessing function
def preprocess_image(image_data):
    """
    Process image data for input to the model
    """
    try:
        # Convert base64 to image if it's in that format
        if isinstance(image_data, str) and image_data.startswith('data:image'):
            # Extract the actual base64 content
            image_data = image_data.split(',')[1]
            image = Image.open(io.BytesIO(base64.b64decode(image_data)))
        elif isinstance(image_data, bytes):
            image = Image.open(io.BytesIO(image_data))
        else:
            image = Image.open(image_data)

        # Apply transformations needed for your model
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

        return transform(image).unsqueeze(0)  # Add batch dimension
    except Exception as e:
        print(f"Error preprocessing image: {e}")
        return None

In [ ]:
# Custom EfficientNet model class to match your saved model structure
import torch.nn as nn  # Import the necessary module

class CustomEfficientNet(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomEfficientNet, self).__init__()
        # Load a pre-trained model with standard structure for feature extraction
        try:
            import timm
            self.model = timm.create_model('efficientnet_b0', pretrained=False)
            # Replace classifier with our own
            self.model.classifier = nn.Linear(self.model.classifier.in_features, num_classes)
        except ImportError:
            # Fallback if timm is not available
            import torchvision.models as models
            self.model = models.efficientnet_b0(pretrained=False)
            self.model.classifier = nn.Linear(self.model.classifier[1].in_features, num_classes)

    def forward(self, x):
        return self.model(x)

In [ ]:
# Load model with custom loading function
def load_model(model_path, num_classes=10):
    """
    Load the saved model using a custom loading approach
    """
    try:
        # Create model instance
        model = CustomEfficientNet(num_classes=num_classes)

        # Load state dict
        state_dict = torch.load(model_path, map_location=torch.device('cpu'))

        # If you still have issues with key names, you may need to map them
        # This is a simplified example - you might need to expand this mapping
        new_state_dict = {}

        # Print first few keys to help debug
        print("First few keys in the state dict:")
        for i, key in enumerate(list(state_dict.keys())[:5]):
            print(f"  {key}")

        # Option 1: Try to load with strict=False
        model.load_state_dict(state_dict, strict=False)

        # Option 2: If that doesn't work, implement a custom mapping
        # This would require analyzing your model's actual structure
        # and mapping the keys appropriately

        model.eval()
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        print("\nAttempting fallback method...")
        return fallback_model(model_path, num_classes)

In [ ]:
def fallback_model(model_path, num_classes=10):
    """
    Fallback method for model loading
    """
    try:
        # Create a simpler model for image classification
        import torchvision.models as models
        model = models.resnet18(pretrained=False)
        model.fc = nn.Linear(model.fc.in_features, num_classes)

        print("Using ResNet18 fallback model. Note: This will NOT use your trained weights.")
        print("Images will be classified using a generic model.")

        model.eval()
        return model
    except Exception as e:
        print(f"Error creating fallback model: {e}")
        return None

In [ ]:
# Function to classify image
def classify_image(model, image_tensor, class_names):
    """
    Classify image using the loaded model
    """
    try:
        if model is None:
            return [{"class": "Unknown", "confidence": "N/A - Model not loaded"}]

        with torch.no_grad():
            outputs = model(image_tensor)
            confidence = torch.nn.functional.softmax(outputs, dim=1)[0]

            # Get top 3 predictions or fewer if we have fewer classes
            top_count = min(3, len(class_names))
            top_conf, top_preds = torch.topk(confidence, top_count)

            results = []
            for i in range(top_count):
                class_id = top_preds[i].item()
                class_name = class_names[class_id] if class_id < len(class_names) else f"Class {class_id}"
                confidence = top_conf[i].item() * 100
                results.append({
                    "class": class_name,
                    "confidence": f"{confidence:.2f}%"
                })

            return results
    except Exception as e:
        print(f"Error during classification: {e}")
        return [{"class": "Error", "confidence": f"Classification failed: {str(e)}"}]

In [ ]:
EFFICIENTNET_PATH = "./skin_cancer_model.pth"  # Update this path
model = load_model(EFFICIENTNET_PATH)

# Define your class names
class_names = [
'akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']


First few keys in the state dict:
  conv_stem.weight
  bn1.weight
  bn1.bias
  bn1.running_mean
  bn1.running_var


In [ ]:
# Configure quantization for loading the model
if device == "cuda":
    # Use 4-bit quantization for GPU
        bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto"
    )
else:
    # For CPU, we'll use 8-bit quantization or standard loading
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map={"": device},
           # offload_folder="./offload",  # this is the fix
            low_cpu_mem_usage=True
        )
    except Exception as e:
        print(f"Error loading quantized model: {e}")
        print("Falling back to standard loading...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
# # Set padding token
# if tokenizer.pad_token is None:
#     tokenizer.pad_token = tokenizer.eos_token

# Create LLM inference pipeline
# llm_pipeline = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=512,
#     do_sample=True,
#     temperature=0.7,
#     top_p=0.95
# )

Device set to use cpu


In [ ]:
def tinyllama_llm(prompt):
    """Generate responses using TinyLlama model"""
    print("In original function")
    try:
        # Format input for TinyLlama chat model
        formatted_prompt = f"<human>: {prompt}\n<assistant>:"
        response = llm_pipeline(formatted_prompt)[0]["generated_text"]

        # Extract only the assistant's response
        assistant_response = response.split("<assistant>:", 1)[1].strip()

        # Clean up any potential trailing tokens
        if "<human>" in assistant_response:
            assistant_response = assistant_response.split("<human>")[0].strip()

        return assistant_response
    except Exception as e:
        print(f"TinyLlama error: {e}")
        return "I couldn't generate a response. Please try again."


In [ ]:
# Load dataset
# For demonstration, create a sample dataframe with HAM10000 structure
print("Loading and preparing dataset...")
sample_data = {
    'dx': ['mel', 'nv', 'bcc', 'akiec', 'vasc', 'bkl', 'df'],
    'dx_type': ['histopathology', 'confocal', 'consensus', 'histopathology', 'follow-up', 'histopathology', 'consensus'],
    'localization': ['scalp', 'back', 'face', 'chest', 'arm', 'leg', 'neck'],
    'age': [45, 32, 67, 55, 40, 62, 28],
    'sex': ['male', 'female', 'male', 'female', 'male', 'female', 'male']
}

try:
    # Try to load the real dataset if available
    metadata = pd.read_csv('/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')
    print("Loaded real HAM10000 dataset")
except:
    # Use sample data if real dataset is not available
    metadata = pd.DataFrame(sample_data)
    print("Using sample dataset")


Loading and preparing dataset...
Loaded real HAM10000 dataset


In [ ]:
# Create a text description field
def create_description(row):
    return f"Diagnosis: {row['dx']}, Type: {row['dx_type']}, Located on {row['localization']}, Age: {row['age']}, Sex: {row['sex']}"

In [ ]:
metadata['text_data'] = metadata.apply(create_description, axis=1)

# Add condition descriptions
condition_info = {
    'mel': 'Melanoma - a serious form of skin cancer that develops from melanocytes',
    'nv': 'Melanocytic nevus - a common mole, a benign growth of melanocytes',
    'bcc': 'Basal cell carcinoma - the most common form of skin cancer, rarely metastasizes',
    'akiec': 'Actinic keratosis - a precancerous skin condition caused by sun damage',
    'vasc': 'Vascular lesion - abnormal blood vessel formation in the skin',
    'bkl': 'Benign keratosis - a non-cancerous growth including seborrheic keratosis',
    'df': 'Dermatofibroma - a benign skin tumor composed of fibroblastic cells'
}

# Add detailed descriptions to the metadata
metadata['detailed_info'] = metadata['dx'].map(lambda x: condition_info.get(x, 'Unknown condition'))
metadata['full_text'] = metadata.apply(lambda row: f"{row['text_data']}. {row['detailed_info']}", axis=1)

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# ✅ This loads index, docs, and internal ID mappings
vectorstore = FAISS.load_local(
    "/content/faiss_store/faiss_store",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True  # ✅ Add this
)

retriever = vectorstore.as_retriever()

<ipython-input-21-7c8a3414eb49>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [ ]:
def get_fallback_response():
    return """I can provide information about skin conditions like:
- Melanoma (mel): A serious form of skin cancer that develops from melanocytes
- Melanocytic nevus (nv): A common mole, a benign growth of melanocytes
- Basal cell carcinoma (bcc): The most common form of skin cancer, rarely metastasizes
- Actinic keratosis (akiec): A precancerous skin condition caused by sun damage
- Vascular lesion (vasc): Abnormal blood vessel formation in the skin
- Benign keratosis (bkl): A non-cancerous growth including seborrheic keratosis
- Dermatofibroma (df): A benign skin tumor composed of fibroblastic cells

What would you like to know about these conditions?"""

In [ ]:
# Define retrieval tool with error handling
def retrieval_tool(query):
    try:
        return retriever.invoke(query)
    except Exception as e:
        print(f"Retrieval error: {e}")
        return []

In [ ]:
# # Define chatbot logic
# def chatbot_with_retrieval(state):
#     # Check if we have any messages
#     if not state["messages"]:
#         # If there are no messages, give a welcome message
#         welcome_msg = "Welcome to the Medical Chatbot! I can provide information about various skin conditions and analyze dermatological data. How can I assist you today?"
#         return {"messages": [AIMessage(content=welcome_msg)]}

#     # Get the latest user message
#     user_msg = None
#     for msg in reversed(state["messages"]):
#         if isinstance(msg, HumanMessage):
#             user_msg = msg
#             break

#     if not user_msg:
#         return {"messages": state["messages"] + [AIMessage(content="I'm not sure I understand. Could you please ask a question about skin conditions?")]}

#     user_query = user_msg.content
#     retrieved_docs = retrieval_tool(user_query)

#     if not retrieved_docs:
#         # Use fallback response if no documents retrieved
#         llm_response = get_fallback_response()
#     else:
#         # Format the retrieved context
#         context_parts = []
#         for i, doc in enumerate(retrieved_docs, 1):
#             context_parts.append(f"Document {i}: {doc.page_content}")

#         context = "\n\n".join(context_parts)
#         prompt = f"""
# Based on the following medical information about skin conditions, please answer the user's query. Try to provide information about the condition based on the available data.
# If you don't have enough information, acknowledge that and provide general guidance.

# RETRIEVED INFORMATION:
# {context}

# USER QUERY: {user_query}

# Your response should be informative, accurate, and empathetic. Do not include any reference numbers or document numbers in your answer.
# """
#         llm_response = tinyllama_llm(prompt)

#     return {"messages": state["messages"] + [AIMessage(content=llm_response)]}

# Update the chatbot function to handle images
def chatbot_with_retrieval(state):
    # Check if we have any messages
    if not state["messages"]:
        # If there are no messages, give a welcome message
        welcome_msg = "Welcome to the Medical Chatbot! I can provide information about various skin conditions and analyze dermatological images. How can I assist you today?"
        return {"messages": [AIMessage(content=welcome_msg)]}

    # Get the latest user message
    user_msg = None
    for msg in reversed(state["messages"]):
        if isinstance(msg, HumanMessage):
            user_msg = msg
            break

    if not user_msg:
        return {"messages": state["messages"] + [AIMessage(content="I'm not sure I understand. Could you please ask a question or upload an image of a skin condition?")]}

    # Check if the message contains an image
    has_image = False
    image_data = None
    user_query = user_msg.content

    # Handle different message types depending on your frontend
    if hasattr(user_msg, 'content') and isinstance(user_msg.content, list):
        # Handle multimodal messages where content is a list of different parts
        text_parts = []
        for part in user_msg.content:
            if part.get('type') == 'text':
                text_parts.append(part.get('text', ''))
            elif part.get('type') == 'image_url' and 'url' in part:
                has_image = True
                image_data = part['url']

        user_query = ' '.join(text_parts)

    # If the user shared an image
    if has_image and image_data and model:
        processed_image = preprocess_image(image_data)
        if processed_image is not None:
            # Classify the image
            results = classify_image(model, processed_image, class_names)

            if results:
                # Create response with classification results
                response = "I've analyzed the skin condition image and here are my findings:\n\n"
                for i, result in enumerate(results, 1):
                    response += f"{i}. {result['class']} - {result['confidence']} confidence\n"

                # Add some general information about the top condition
                top_condition = results[0]['class']
                retrieved_docs = retrieval_tool(top_condition)

                if retrieved_docs:
                    context_parts = []
                    for i, doc in enumerate(retrieved_docs, 1):
                        context_parts.append(f"{doc.page_content}")
                    context = "\n\n".join(context_parts)

                    prompt = f"""
                    Based on the image analysis, the most likely condition is {top_condition}.

                    Here's relevant information about this condition:
                    {context}

                    The user query was: {user_query if user_query else "Image analysis"}

                    Provide a helpful, informative response about this condition based on the image analysis and retrieved information.
                    Don't mention document numbers or reference numbers.
                    """

                    detailed_response = tinyllama_llm(prompt)
                    response += f"\n\n{detailed_response}"
                else:
                    response += f"\n\nI identified the top condition as {top_condition}, but I don't have detailed information about it in my knowledge base. I recommend consulting with a dermatologist for a proper diagnosis."

                return {"messages": state["messages"] + [AIMessage(content=response)]}
            else:
                return {"messages": state["messages"] + [AIMessage(content="I couldn't analyze the image properly. Please make sure it's a clear image of the skin condition and try again.")]}

    # If no image or image processing failed, use the text-based approach
    retrieved_docs = retrieval_tool(user_query)

    if not retrieved_docs:
        # Use fallback response if no documents retrieved
        llm_response = get_fallback_response()
    else:
        # Format the retrieved context
        context_parts = []
        for i, doc in enumerate(retrieved_docs, 1):
            context_parts.append(f"Document {i}: {doc.page_content}")
        context = "\n\n".join(context_parts)

        prompt = f"""
        Based on the following medical information about skin conditions, please answer the user's query.
        Try to provide information about the condition based on the available data.
        If you don't have enough information, acknowledge that and provide general guidance.

        RETRIEVED INFORMATION:
        {context}

        USER QUERY:
        {user_query}

        Your response should be informative, accurate, and empathetic.
        Do not include any reference numbers or document numbers in your answer.
        """

        llm_response = tinyllama_llm(prompt)

    return {"messages": state["messages"] + [AIMessage(content=llm_response)]}

In [ ]:
# def human_input(state):
#     # Print the last AI message
#     for msg in reversed(state["messages"]):
#         if isinstance(msg, AIMessage):
#             print(f"Bot: {msg.content}")
#             break

#     user_input = input("User: ")
#     if user_input.lower() in {"exit", "quit", "bye"}:
#         return {"finished": True, "messages": state["messages"] + [HumanMessage(content=user_input)]}

#     return {"messages": state["messages"] + [HumanMessage(content=user_input)]}
# Update your human input function to handle image uploads

import ipywidgets as widgets
from IPython.display import display, clear_output
from langchain.schema import HumanMessage, AIMessage
import time

from IPython.display import display, clear_output
import ipywidgets as widgets
from langchain.schema import HumanMessage
import io
import base64
from PIL import Image

from IPython.display import display, clear_output
from google.colab import files
import io
from langchain.schema import HumanMessage
import time

def handle_file_upload(state):
    """Use Colab's built-in file uploader instead of widgets"""

    print("Please upload an image file when prompted...")

    # Use Colab's built-in upload function
    uploaded = files.upload()

    if not uploaded:
        print("No file was uploaded")
        return {"messages": state["messages"]}

    # Process the first uploaded file
    for filename, content in uploaded.items():
        print(f"Processing {filename} ({len(content)} bytes)...")

        # Add to messages
        state["messages"].append(HumanMessage(content=[
            {"type": "text", "text": "Please analyze this image."},
            {"type": "image_bytes", "bytes": content}
        ]))

        print("✅ Image uploaded successfully!")
        break  # Only process the first file

    return {"messages": state["messages"]}


def human_input(state):
    # Print the last AI message
    for msg in reversed(state["messages"]):
        if isinstance(msg, AIMessage):
            print(f"Bot: {msg.content}")
            break

    # In a real frontend, you'd handle image upload differently
    # This is a simplified CLI example
    print("Options: [1] Ask a question [2] Upload an image [3] Exit")
    choice = input("Choice: ")

    if choice == "1":
        user_input = input("Your question: ")
        return {"messages": state["messages"] + [HumanMessage(content=user_input)]}
    elif choice == "2":
        return handle_file_upload(state)
    elif choice == "3" or choice.lower() in {"exit", "quit", "bye"}:
        return {"finished": True, "messages": state["messages"] + [HumanMessage(content="bye")]}
    else:
        print("Invalid choice, please try again.")
        return human_input(state)


def maybe_exit(state):
    # Fix: Return the actual node name "chatbot" if not finished
    return END if state.get("finished", False) else "chatbot"

In [ ]:
# # For images
# from langchain.agents import Tool

# def classify_skin_image(image_path: str) -> str:
#     # Call your EfficientNet model here
#     return predict_image(image_path)  # returns label like "mel"

# image_tool = Tool.from_function(
#     func=classify_skin_image,
#     name="classify_skin_image",
#     description="Classifies a skin image and returns the predicted condition."
# )

In [ ]:
# def image_input_node(state):
#     # Get the last message
#     for msg in reversed(state["messages"]):
#         if isinstance(msg, HumanMessage):
#             if msg.content.endswith((".jpg", ".jpeg", ".png")) and os.path.exists(msg.content):
#                 prediction = classify_skin_image(msg.content)
#                 return {"messages": state["messages"] + [AIMessage(content=f"Image diagnosed as: {prediction}")], "diagnosis": prediction}
#     return {"messages": state["messages"] + [AIMessage(content="Please upload a valid image path.")]}

# graph_builder.add_node("classify_image", image_input_node)


In [ ]:
# # Build conversation graph
# print("Building conversation graph...")
# graph_builder = StateGraph(dict)
# graph_builder.add_node("chatbot", chatbot_with_retrieval)
# graph_builder.add_node("human", human_input)
# graph_builder.add_edge(START, "chatbot")
# graph_builder.add_edge("chatbot", "human")
# graph_builder.add_conditional_edges("human", maybe_exit)
# chat_graph = graph_builder.compile()

print("Building conversation graph...")
graph_builder = StateGraph(dict)
graph_builder.add_node("chatbot", chatbot_with_retrieval)
graph_builder.add_node("human", human_input)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", "human")
# graph_builder.add_conditional_edges("human", maybe_exit)
chat_graph = graph_builder.compile()

Building conversation graph...


In [ ]:
# QLoRA Fine-tuning function
def fine_tune_model_with_qlora(training_data, model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0", output_dir="./finetuned_model"):
    """
    Fine-tune TinyLlama model using QLoRA

    Args:
        training_data: List of dictionaries with 'instruction' and 'response' keys
        model_name: Base model to fine-tune
        output_dir: Directory to save fine-tuned model
    """
    print(f"Starting QLoRA fine-tuning process for {model_name}...")

    # Format the training data
    def format_instruction(example):
        return f"<human>: {example['instruction']}\n<assistant>: {example['response']}"

    formatted_data = [format_instruction(item) for item in training_data]

    # Create a dataset
    dataset = Dataset.from_dict({"text": formatted_data})

    # Tokenize the dataset
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

    tokenized_dataset = dataset.map(tokenize_function, batched=True)

    # Configure LoRA
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,  # Rank
        lora_alpha=32,  # Alpha parameter for LoRA scaling
        lora_dropout=0.1,  # Dropout probability for LoRA layers
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # Target attention modules for adaptation
    )

    # Prepare model for training
    if device == "cuda":
        # For GPU fine-tuning
        model_to_train = prepare_model_for_kbit_training(model)
        model_to_train = get_peft_model(model_to_train, peft_config)
    else:
        # For CPU fine-tuning (simpler configuration)
        model_to_train = get_peft_model(model, peft_config)

    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=4 if device == "cuda" else 1,  # Smaller batch size for CPU
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=device == "cuda",  # Use fp16 only on GPU
        logging_steps=10,
        save_steps=200,
        save_total_limit=3,
    )

    # Initialize trainer
    trainer = Trainer(
        model=model_to_train,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    )

    # Train the model
    print("Starting training...")
    trainer.train()

    # Save the fine-tuned model
    trainer.save_model(output_dir)

    print(f"Model fine-tuned and saved to {output_dir}")
    return output_dir


In [ ]:
# Example training data for fine-tuning
def prepare_training_data():
    """Prepare training data from HAM10000 dataset for fine-tuning"""
    # Create instructions based on skin conditions
    training_samples = []

    # Create general questions about skin conditions
    for condition, description in condition_info.items():
        # Question about symptoms
        training_samples.append({
            "instruction": f"What are the symptoms of {condition}?",
            "response": f"For {condition} ({description}), typical symptoms include changes in skin appearance, " +
                       f"such as color changes, irregular borders, and asymmetry. This condition is often found on " +
                       f"areas like the face, neck, and sun-exposed areas. Always consult a dermatologist for diagnosis."
        })

        # Question about treatments
        training_samples.append({
            "instruction": f"How is {condition} treated?",
            "response": f"Treatment for {condition} ({description}) depends on the severity and location. " +
                       f"Options may include topical medications, surgical removal, cryotherapy, or observation. " +
                       f"A dermatologist should determine the best treatment approach based on individual factors."
        })

    # Create questions about metadata
    for i, row in metadata.head(5).iterrows():
        training_samples.append({
            "instruction": f"Tell me about a {row['dx']} case.",
            "response": f"Here's information about a {row['dx']} case: {row['full_text']} " +
                       f"This condition requires proper medical evaluation for diagnosis and treatment."
        })

    # Add general skin health questions
    training_samples.extend([
        {
            "instruction": "What factors increase the risk of skin cancer?",
            "response": "Factors that increase skin cancer risk include excessive UV exposure (sun or tanning beds), " +
                       "fair skin, history of sunburns, family history of skin cancer, weakened immune system, and " +
                       "advanced age. Regular skin examinations and sun protection can help reduce these risks."
        },
        {
            "instruction": "How can I prevent melanoma?",
            "response": "To prevent melanoma: limit sun exposure especially during peak hours (10am-4pm), " +
                       "wear broad-spectrum sunscreen (SPF 30+), wear protective clothing, avoid tanning beds, " +
                       "conduct regular skin self-exams, and see a dermatologist annually, especially if you " +
                       "have risk factors like fair skin or family history."
        }
    ])

    return training_samples


In [ ]:
# # Define function to start the chat session
# def start_chat():
#     print("Medical Chatbot Ready! Type 'exit' to quit.")
#     try:
#         state = chat_graph.invoke({"messages": []})
#     except Exception as e:
#         print(f"Error starting chatbot: {e}")
#         print("\nTroubleshooting tips:")
#         print("1. Check if you have properly installed all required packages")
#         print("2. Make sure you have enough memory available")
#         print("3. Try reducing model complexity or batch size")
# Define function to start the chat session
def start_chat():
    print("Medical Chatbot Ready! Type 'exit' to quit.")
    try:
        # Enable debug mode to see what's happening
        # set_debug(True)
        # Initialize with empty messages and run until completion
        state = {"messages": []}
        while True:
            # Run the graph once
            state = chat_graph.invoke(state)
            # Check if we're finished
            if state.get("finished", False):
                print("Chat session ended.")
                break
    except Exception as e:
        print(f"Error in chat session: {e}")
        print("\nTroubleshooting tips:")
        print("1. Check if you have properly installed all required packages")
        print("2. Make sure you have enough memory available")
        print("3. Try reducing model complexity or batch size")
        print("4. Check for compatibility issues with your model format")

In [ ]:
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
from google.colab import files
import io
import base64
from PIL import Image
from langchain.schema import HumanMessage, AIMessage

def create_chatbot_ui():
    """Create a user interface for the medical chatbot"""

    # CSS for better appearance
    display(HTML("""
    <style>
    .chat-container {
        max-height: 400px;
        overflow-y: auto;
        border: 1px solid #ddd;
        padding: 10px;
        margin-bottom: 10px;
        background-color: #f9f9f9;
        border-radius: 5px;
    }
    .user-message {
        background-color: #e3f2fd;
        padding: 8px 12px;
        border-radius: 15px;
        margin: 5px 0;
        max-width: 80%;
        margin-left: auto;
        text-align: right;
    }
    .bot-message {
        background-color: #f1f1f1;
        padding: 8px 12px;
        border-radius: 15px;
        margin: 5px 0;
        max-width: 80%;
    }
    .message-container {
        display: flex;
        flex-direction: column;
        margin-bottom: 10px;
    }
    </style>
    """))

    # Chat container - where messages will appear
    chat_output = widgets.Output(layout=widgets.Layout(height='400px', overflow_y='auto'))

    # Text input for user messages
    text_input = widgets.Text(
        value='',
        placeholder='Type your message here...',
        description='',
        layout=widgets.Layout(width='70%')
    )

    # Send button for text messages
    send_button = widgets.Button(
        description='Send',
        button_style='primary',
        layout=widgets.Layout(width='15%')
    )

    # Upload button for images
    upload_button = widgets.Button(
        description='Upload Image',
        button_style='info',
        layout=widgets.Layout(width='15%')
    )

    # Create layout
    input_box = widgets.HBox([text_input, send_button, upload_button])

    # Define message display functions first (before using them)
    def print_user_message(message):
        with chat_output:
            display(HTML(f'<div class="message-container"><div class="user-message">{message}</div></div>'))

    def print_bot_message(message):
        with chat_output:
            display(HTML(f'<div class="message-container"><div class="bot-message">{message}</div></div>'))

    def print_image(img_data):
        with chat_output:
            img_b64 = base64.b64encode(img_data).decode('utf-8')
            display(HTML(f'<div class="message-container"><div class="user-message"><img src="data:image/jpeg;base64,{img_b64}" style="max-width:200px; border-radius:5px;" /></div></div>'))

    # Initialize state and display components
    display(chat_output)
    display(input_box)

    # Initialize state with empty messages
    state = {"messages": []}

    # Handler for text messages
    def handle_text_message(b=None):
        message = text_input.value
        if not message.strip():
            return

        # Clear input
        text_input.value = ''

        # Exit command
        if message.lower() == 'exit':
            print_bot_message("Chat session ended.")
            state["finished"] = True
            return

        # Display user message
        print_user_message(message)

        # Add message to state
        state["messages"].append(HumanMessage(content=message))

        # Process with chatbot_with_retrieval function
        try:
            # Call the chatbot function directly with the current state
            new_state = chatbot_with_retrieval(state)

            # Update state
            state.update(new_state)

            # Find the last AI message to display
            for msg in reversed(state["messages"]):
                if isinstance(msg, AIMessage):
                    print_bot_message(msg.content)
                    break
        except Exception as e:
            print_bot_message(f"Error: {str(e)}")

    # Handler for image uploads
    def handle_image_upload(b):
        print_bot_message("Please select an image of the skin condition...")

        # Use Colab's upload function
        uploaded = files.upload()

        if not uploaded:
            print_bot_message("No image was uploaded.")
            return

        # Process uploaded image
        for filename, content in uploaded.items():
            try:
                # Display the image in chat
                print_image(content)
                print_bot_message(f"Analyzing image: {filename}...")

                # Add to messages with multi-modal content
                state["messages"].append(HumanMessage(content=[
                    {"type": "text", "text": "Please analyze this skin condition."},
                    {"type": "image_bytes", "bytes": content}
                ]))

                # Process with chatbot function
                new_state = chatbot_with_retrieval(state)

                # Update state
                state.update(new_state)

                # Find the last AI message to display
                for msg in reversed(state["messages"]):
                    if isinstance(msg, AIMessage):
                        print_bot_message(msg.content)
                        break

                break  # Only process first image
            except Exception as e:
                print_bot_message(f"Error processing image: {str(e)}")

    # Connect handlers to buttons
    send_button.on_click(handle_text_message)
    upload_button.on_click(handle_image_upload)

    # Handle Enter key in text input
    text_input.on_submit(handle_text_message)

    # Show initial welcome message - moved here after print_bot_message is defined
    print_bot_message("Welcome to the Medical Chatbot! How can I help you with your skin condition today?")

    # Start the chatbot by invoking it with empty messages to get welcome message
    if not state["messages"]:
        new_state = chatbot_with_retrieval(state)
        state.update(new_state)

        # If we got a welcome message, display it
        for msg in state["messages"]:
            if isinstance(msg, AIMessage):
                print_bot_message(msg.content)
                break

    return {
        'chat_output': chat_output,
        'input': text_input,
        'send_button': send_button,
        'upload_button': upload_button,
        'state': state
    }

# Updated start_chat function that uses the UI
def start_chat():
    print("Medical Chatbot Ready!")

    try:
        # Display UI and let it handle the interaction
        ui = create_chatbot_ui()

        # No need for the while loop from the original function
        # as the UI buttons now handle the interaction flow

    except Exception as e:
        print(f"Error in chat session: {e}")
        print("\nTroubleshooting tips:")
        print("1. Check if you have properly installed all required packages")
        print("2. Make sure you have enough memory available")
        print("3. Try reducing model complexity or batch size")
        print("4. Check for compatibility issues with your model format")

## If you're using the finetuned_model folder then run this (unzip the folder first)


In [ ]:
!mkdir -p ./offload

In [ ]:
if __name__ == "__main__":
    fine_tuned_dir = "/content/drive/MyDrive/finetuned_model"

    # Load EfficientNet model for image classification
    EFFICIENTNET_PATH = "/content/drive/MyDrive/skin_cancer_model.pth"  # Update this path
    try:
        print(f"Loading image classification model from {EFFICIENTNET_PATH}...")
        model = load_model(EFFICIENTNET_PATH)
        print("Image classification model loaded successfully!")
    except Exception as e:
        print(f"Error loading image classification model: {e}")
        print("Image classification may not work correctly.")
        model = None

    # Load fine-tuned LLM model
    try:
        print(f"Loading fine-tuned text model from {fine_tuned_dir}...")

        # Load base model first (same config you used during training)
        tokenizer = AutoTokenizer.from_pretrained(fine_tuned_dir)
        base_model = AutoModelForCausalLM.from_pretrained(
            fine_tuned_dir,
            device_map="cpu",  # or {"": "cpu"} if you're on CPU
            offload_folder="./offload",  # optional if low RAM
            low_cpu_mem_usage=True
        )

        # Apply PEFT adapter (if you used QLoRA or PEFT)
        model = PeftModel.from_pretrained(base_model, fine_tuned_dir)

        # # Create generation pipeline and assign to tinyllama_llm function
        # # llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer,
        # #                         max_new_tokens=512, do_sample=True)
        # llm_pipeline = pipeline(
        #           "text-generation",
        #           model=model,
        #           tokenizer=tokenizer,
        #           max_new_tokens=512,
        #           do_sample=True,
        #           temperature=0.7,
        #           top_p=0.95
        #       )

        # Create a wrapper function that matches your existing code's expectations
        # def tinyllama_llm(prompt):
        #     try:
        #         outputs = llm_pipeline(prompt)
        #         return outputs[0]['generated_text'].replace(prompt, "").strip()
        #     except Exception as e:
        #         print(f"Error generating text: {e}")
        #         return "I apologize, but I'm having trouble generating a response right now."

        # def tinyllama_llm(prompt):
        #   """Generate responses using TinyLlama model"""
        #   # print("In original function")
        #   try:
        #       # Format input for TinyLlama chat model
        #       formatted_prompt = f"<human>: {prompt}\n<assistant>:"
        #       response = llm_pipeline(formatted_prompt)[0]["generated_text"]

        #       # Extract only the assistant's response
        #       assistant_response = response.split("<assistant>:", 1)[1].strip()

        #       # Clean up any potential trailing tokens
        #       if "<human>" in assistant_response:
        #           assistant_response = assistant_response.split("<human>")[0].strip()

        #       return assistant_response
        #   except Exception as e:
        #       print(f"TinyLlama error: {e}")
        #       return "I couldn't generate a response. Please try again."

        def tinyllama_llm(prompt):
          """Generate responses using TinyLlama model (QLoRA-compatible)"""
          try:
              # Format input for TinyLlama chat model
              formatted_prompt = f"<human>: {prompt}\n<assistant>:"

              # Tokenize the input
              inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

              # Generate output
              outputs = model.generate(
                  **inputs,
                  max_new_tokens=512,
                  do_sample=True,
                  temperature=0.7,
                  top_p=0.95,
                  pad_token_id=tokenizer.eos_token_id,
              )

              # Decode and clean response
              generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
              assistant_response = generated_text.split("<assistant>:", 1)[1].strip()

              if "<human>" in assistant_response:
                  assistant_response = assistant_response.split("<human>")[0].strip()

              return assistant_response

          except Exception as e:
              print(f"TinyLlama error: {e}")
              return "I couldn't generate a response. Please try again."




        print("Fine-tuned text model loaded successfully!")
    except Exception as e:
        print(f"Error loading fine-tuned text model: {e}")
        print("Using fallback text generation...")

        # Simple fallback text generation function
        def tinyllama_llm(prompt):
            return "I apologize, but my text generation model is not currently available. I can only analyze images at the moment."

    # Start chat
    start_chat()

Loading image classification model from /content/drive/MyDrive/skin_cancer_model.pth...
First few keys in the state dict:
  conv_stem.weight
  bn1.weight
  bn1.bias
  bn1.running_mean
  bn1.running_var
Image classification model loaded successfully!
Loading fine-tuned text model from /content/drive/MyDrive/finetuned_model...
Fine-tuned text model loaded successfully!
Medical Chatbot Ready!


Output(layout=Layout(height='400px', overflow_y='auto'))

Saving melanoma.jpeg to melanoma (1).jpeg


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Saving the model (if you train it)

# EVALUATION

In [ ]:
Evaluation Results:
rouge1: 0.2936
rouge2: 0.0754
rougeL: 0.2391
bleu: 0.0271
semantic_similarity: 0.7751